#6-1. LightningModule Class
<hr>

Pytorch lightning에서는 trainer와 모델이 상호작용을 할 수 있도록 pytorch의 nn.Module의 상위 클래스인 lightning module을 구현해야 합니다. 기존 PyTorch는 DataLoader, Mode, optimizer, Training loof 등을 전부 따로따로 코드로 구현을 해야하는데 Pytorch Lightning에서는 Lightning Model class 안에 이 모든것을 한번에 구현하도록 되어있습니다.

ligthning module을 정의하기 위해 LightningModule 클래스를 상속받고 모델, training, validation, test 루프 그리고 optimizer 등을 구현해야 합니다.

Training 루프는 training_step 메소드에 있고, validation 루프는 validation_step 메소드에 들어가있습니다. 메트릭의 일반적인 리포팅은 validation_epoch_end 메소드에 있습니다. Model 클래스 안에는, training_step과 validation_step 모두 배치(batch)에서 x와 y를 가져오기 위해 step 메소드를 호출합니다. 또한 forward pass와 loss를 리턴하기 위해 foward를 호출합니다. Training이 끝나면, 우리의 validation 루프가 호출되고 epoch이 끝날 때 validation_epoch_end가 호출되서 결과가 누적되고 score가 계산됩니다.



In [ ]:
class Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            ...
        )

    def forward(self, x):
        pass

    def training_step(self, batch, batch_idx):
        pass

    def validation_step(self, batch, batch_idx):
        pass

    def test_step(self, batch, batch_idx):
        pass

    def configure_optimizers(self):
        pass


Lightning Module은 6가지로 구성됩니다.

* Computations (init).
* Train loop (training_step)
* Validation loop (validation_step)
* Test loop (test_step)
* Prediction loop (predict_step)
* Optimizers (configure_optimizers)


## init
init 는 초기화 메서드 입니다. Lightning Module class에서 사용할 신경망을 정의 합니다. Pytorch에 신경망 Layer를 생성하려면, torch.nn.module에서 불러오거나 확장해야 합니다.

## forward
forward은 Pytorch에서처럼 추론에 사용됩니다. forward는 모델의 추론 결과를 제공하고 싶을 때 사용합니다.



In [1]:
def forward(self, x):
  return self.model(x)

## training_step
training_step 은 nn.Module의 forward와 유사하지만, 단일 배치에서의 손실을 반환해야 하며, 이는 train loop로 자동 반복됩니다. training_step은 학습 루프의 body 부분을 나타냅니다. 이 메소드에서는 argument로 training 데이터로더가 제공하는 batch와 해당 batch의 인덱스가 주어지고 학습 로스를 계산하여 리턴합니다. pytorch lightning은 편리하게도 batch의 텐서를 cpu 혹은 gpu 텐서로 변경하는 코드를 따로 추가하지 않아도 trainer의 설정에 따라 자동으로 적절한 타입으로 변경해줍니다.

만약 epoch-level metric을 계산하고 log를 하려면 .log 메서드를 사용합니다. 만약에 각 training_step의 결과로 무엇인가 할 일이 있으면 training_epoch_end 메서드에 작성합니다.

log()개체가 자동으로 complete epoch와 device에서 요청된 metrics(통계)를 줄일 수 있습니다.



In [ ]:
def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    # logs metrics for each training_step,
    # and the average across the epoch, to the progress bar and logger
    self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

def training_epoch_end(self, training_step_outputs):
    for pred in training_step_outputs:
        ...

## validation_step
loss 및 metric logging을 위한 validation_step및 test_step을 추가할 수 있습니다. validation_step은 학습 중간에 모델의 성능을 체크하는 용도로 사용합니다. training_step과 마찬가지로 validation 데이터로더에서 제공하는 배치를 가지고 확인하고자 하는 통계량을 기록할 수 있습니다.

만약에 각 validation_step의 결과로 무엇인가 할 일이 있으면 validation_epoch_end 메서드에 작성합니다.



In [ ]:
def validation_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    self.log("val_loss", loss)
    pred = ...
    return pred

def validation_epoch_end(self, validation_step_outputs):
    for pred in validation_step_outputs:
        ...


## test_step
test_step은 앞의 두 함수와 비슷하게 test 데이터로더에서 제공하는 배치를 가지고 확인하고 싶은 통계량을 기록하는데 사용할 수 있습니다. test loop 코드는 validation loop 코드와 거의 동일합니다. 호출할 때는 test_step()메서드 를 재정의해야 합니다. 테스트 루프는 test()가 사용될 때만 호출된다는 것 입니다.



In [ ]:
# call after training
trainer = Trainer()
trainer.fit(model)

# automatically auto-loads the best weights from the previous run
trainer.test(dataloaders=test_dataloader)

## configure_optimizers
configure_optimizers에서는 모델의 최적 파라미터를 찾을 때 사용할 optimizer와 scheduler를 구현합니다. Optimizer 와 Learning rate scheduler에 대해서는 다음장에 좀더 자세하게 설명하겠습니다.

## Boston House 예제
위의 순서에 맞춰 간단한 Boston 집값 예측 예제를 풀어 보겠습니다.





In [3]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.4 MB/s eta 0:00:00


In [6]:
import torch
import pytorch_lightning as pl
from torch import Tensor, nn
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
import numpy as np

#Boston 집값 데이터를 읽어온다.
X, y = load_boston(return_X_y=True)

class SklearnDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        super().__init__()
        scaler = MinMaxScaler() 

        scaler.fit(X) 
        self.X = scaler.transform(X)
        self.Y = y

    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        x = self.X[idx].astype(np.float32)
        y = self.Y[idx].astype(np.float32)
        return x, y

bostonds = SklearnDataset(X, y)
train_loader = DataLoader(bostonds, batch_size=32, shuffle=True, drop_last=True, )

class LinRegModel(pl.LightningModule):
    def __init__(self, input_dim: int):
        super().__init__()
        self.linear = nn.Linear(in_features=13, out_features=1, bias=True)

    def forward(self, x):
        y_hat = self.linear(x)
        return y_hat

    def training_step(self, batch, batch_idx):
        x, y = batch
        # flatten any input
        x = x.view(x.size(0), -1)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y, reduction="sum")
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

trainer = pl.Trainer()
model = LinRegModel(input_dim=13)
trainer.fit(model, train_loader)

ImportError: ignored